
# Model Functions

First Part based on Tony and Matheus code


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from datetime import datetime
import io

# Tensorflow
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorboard.plugins.hparams import api as hp

#TensorBoard notebook extension and clear any logs from previous runs
%load_ext tensorboard
!rm -rf ./logs/ 

In [4]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [7]:
#for colab running
if True:
  !echo "Downloading 101_Object_Categories for image notebooks"
  !curl -L -o 101_ObjectCategories.tar.gz --progress-bar http://www.vision.caltech.edu/Image_Datasets/Caltech101/101_ObjectCategories.tar.gz
  !tar -xzf 101_ObjectCategories.tar.gz
  !rm 101_ObjectCategories.tar.gz
  !ls

######################################################################## 100.0%
101_ObjectCategories  sample_data


In [6]:
data_path = '101_ObjectCategories'

assert os.path.isdir(data_path), 'Dataset not found 👎🏻'
print ('Dataset found 🤙🏻')


AssertionError: ignored

## Some Functions


*   generate_model
*   load_weights
*   compile_model
*   train_model



In [2]:
def generate_model(hparams, resnet50model =True):

    '''
    A function that generates the model based on the hyperparameters that are given.
    The model is not yet compiled, neither with the weights

    Inputs:
      - resnetmodel: a boolean that is by default True saying that the model use is ResNet50, if you want to use other model pass it in this argument
      - hparams: a dictionnary containning, at least, the following hyperparameters (in parenthesis is the hyperparameter key)
          - number of dense layers ('HP_NUM_DENSEUNITS'), list of numbers
          - number of units in the dense layer ('HP_NUM_UNITS'), list of numbers
          - number of dropout layers ('HP_DROPOUT_UNITS'), list of numbers
          - dropout rate in the dropouts layer ('HP_DROPOUT'),listof numbers lower than 1
          - number of trainable layers ('HP_TRAINABLES'), list of numbers


    Output:
      - a model not compiled but with the hiperparameters chosen

    '''

    #Things that can be added to the function: order of dropout and dense layers, activation function choice

    #model creating and adding dense and dropout layers
    model = Sequential()
    if type(resnet50model)== bool:
        model.add(ResNet50(include_top = False, weights = 'imagenet'))
    else:
        model.add(resnet50model)
    model.add(GlobalAveragePooling2D())
    for i in hparams['HP_DROPOUT_UNITS']:
        model.add(hparams['HP_DROPOUT'])
    for i in hparams['HP_NUM_DENSEUNITS']:
        model.add(Dense(hparams['HP_NUM_UNITS'],activation='relu'))
    model.add(Dense(102,activation='softmax'))

    #model layers trainable 
    model.layers[-5].layers[-1].trainable = False
    for i in model.layers[-5].layers :
        i.trainable = False
    layer_number = -1
    for i in range(hparams['HP_TRAINABLES']):
        layer_number-=i
        model.layers[-5].layers[layer_number].trainable = True

    return model

    

In [5]:
def load_weights(model,weights_filepath):
    '''
    A function that loads weight to the model.
    The function raise an exception if the file is not a .h5

    Inputs:
      - model: a generated keras model
      - weights_filepath: the filepath to weights data, must be a .h5 file

    Output:
      - a model with weights
    '''
    if weights_filepath.endswith('.h5'):
        return model.load_weights('weigths_filepath')
    else:
        print('Is not a .h5 file')

In [6]:
def compile_model(model,hparams,metrics_list):

    '''
    A function that compiles the model according to the hyperparameters defined.

    Inputs:
      - model: a generated keras model
      - hparams: a dictionnary containning, at least, the following hyperparameters (in parenthesis is the hyperparameter key)
          - the loss function ('HP_LOSS_FUNCTION'), list of loss functions types in keras
          - the optimizer ('HP_OPTMIZER'), list of optmizes in keras
      - metrics_list: the metrics for the model

    Output:
      - a model with weights
    '''
    model.compile(loss= hparams['HP_LOSS_FUNCTION'],
                optimizer= hparams['HP_OPTMIZER'],
                metrics= metrics_list)
    return model

In [8]:
def train_model(model, train_generator, validation_generator, hparams, callbacks = False, modelpath = False):
    '''
  A function that trains the model with the training data.
  Can return only the model if no pathway is specified.
  Save the model in a pathway if pathway specified.

  Inputs:
      - model: a generated keras model already compiled
      - train_generator: the train data generated
      - validation_generator: the validation data generated
      - hparams: a dictionnary containning, at least, the following hyperparameters (in parenthesis is the hyperparameter key)
          - number of epochs ('HP_EPOCHS'), list of numbers
      - callback: a boolean. If there is a tensorboard callback and a logdir, specified in the following variables the parameters must be turn True:
          - tensorboard_callback
          - logdir
      - modelpath: False if it is not to save the model. If it is to save it must be specified the pathway as a string is this argument
      
     
          
    
  Output:
      - a model trained
      - a file .h5 with the weights of the model
  '''
    if callbacks:
        model.fit_generator(generator=train_generator,
                        steps_per_epoch=len(train_generator),
                        epochs=hparams['HP_EPOCHS'],
                        validation_data=validation_generator,
                        validation_steps= len(validation_generator),
                        verbose=True,
                        shuffle=True,
                        callbacks=[tensorboard_callback])
    else:
        model.fit_generator(generator=train_generator,
                        steps_per_epoch=len(train_generator),
                        epochs=hparams['HP_EPOCHS'],
                        validation_data=validation_generator,
                        validation_steps= len(validation_generator),
                        verbose=True,
                        shuffle=True)
    if type(modelpath) == bool:
        if not modelpath:
            return model
    else:
        model.save(modelpath)
        print('Saved trained model at %s ' % model_path)
        return model

    
